In [1]:
import pandas as pd
import numpy as np
import random
import pickle

## Creating the phrases datasets
+ getting the dataset
+ (select the ones that have no comments - later delete this part)
+ separate into:
    + original_phrases: the ones that came from Wikipedia
    + ground truths (only the changes):
        + all_female_phrases_gt: the ones where the entire universe is feminine
        + all_male_phrases_gt: the ones where the entire universe is masculine
        + female_sub_phrases_gt: the ones where only what is related to the subject is feminine
        + male_sub_phrases_gt: the ones where only what is related to the subject is masculine
    + complete phrases (change not NaN in ground truths in the original phrases):
        + final_all_fem: the ones where the entire universe is feminine
        + final_all_male: the ones where the entire universe is masculine
        + final_subj_fem: the ones where only what is related to the subject is feminine
        + final_subj_male: the ones where only what is related to the subject is masculine
+ for every line of the changed datasets copy the original words for the NaN places
+ choose randomly the names and surnames and add them to their spots

In [2]:
df_original=pd.read_excel('all_phrases.xlsx',index_col=None, header=None)


/home/martao/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [3]:
df_original

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,NaN,NaN,Original,Shortly,afterwards,",",he,visits,Netherfield,",",...,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,All Female,NaN,NaN,NaN,she,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,All Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,Subject Female,NaN,NaN,NaN,she,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,Subject Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8430,NaN,NaN,Original,His,wife,tends,to,Louisa,",",and,...,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8431,NaN,NaN,All Female,Her,NaN,NaN,NaN,name_female_1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8432,NaN,NaN,All Male,NaN,husband,NaN,NaN,name_male_1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8433,NaN,NaN,Subject Female,NaN,NaN,NaN,NaN,name_female_1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df=df_original.copy()
df=df[df[0].isna()] #here I deleted all the phrases that should be checked

original_phrases=df[df[2]=='Original'].drop([0,1,2],axis=1)

all_female_phrases_gt=df[df[2]=='All Female'].drop([0,1,2],axis=1)
all_male_phrases_gt=df[df[2]=='All Male'].drop([0,1,2],axis=1)
female_sub_phrases_gt=df[df[2]=='Subject Female'].drop([0,1,2],axis=1)
male_sub_phrases_gt=df[df[2]=='Subject Male'].drop([0,1,2],axis=1)

print(int(df_original.shape[0]/5),original_phrases.shape,all_female_phrases_gt.shape,all_male_phrases_gt.shape,female_sub_phrases_gt.shape,male_sub_phrases_gt.shape)

1687 (1610, 29) (1610, 29) (1610, 29) (1610, 29) (1610, 29)


In [5]:
all_female_phrases_gt.index
#original_phrases.index

Int64Index([   1,    6,   11,   16,   21,   26,   31,   36,   41,   46,
            ...
            8381, 8386, 8391, 8396, 8401, 8406, 8411, 8416, 8426, 8431],
           dtype='int64', length=1610)

In [6]:
original_phrases

,3,4,5,6,7,8,9,10,11,12,...,22,23,24,25,26,27,28,29,30,31
0,Shortly,afterwards,",",he,visits,Netherfield,",",Mr.,Bingley,'s,...,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,He,appears,attracted,to,Jane,Bennet,(,the,eldest,Bennet,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,He,declines,to,dance,with,Elizabeth,(,the,second-eldest,Bennet,...,enough,to,tempt,him,.,NaN,NaN,NaN,NaN,NaN
15,He,is,a,pompous,",",obsequious,clergyman,who,intends,to,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,After,learning,that,Jane,may,soon,be,engaged,",",he,...,both,age,and,beauty,.,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8405,Andersen,'s,mother,",",Anne,Marie,Andersdatter,",",was,an,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8410,A,girl,named,Riborg,Voigt,was,the,unrequited,love,of,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8415,Stephen,'s,father,gets,into,debt,and,the,family,leaves,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8425,Her,brother,Charles,notices,that,she,is,less,lively,after,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
all_female_phrases_gt

,3,4,5,6,7,8,9,10,11,12,...,22,23,24,25,26,27,28,29,30,31
1,NaN,NaN,NaN,she,NaN,NaN,NaN,Mrs.,surname_1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,She,NaN,NaN,NaN,name_female_1,surname_1,NaN,NaN,NaN,surname_1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,She,NaN,NaN,NaN,NaN,name_female_1,NaN,NaN,NaN,surname_1,...,NaN,NaN,NaN,her,NaN,NaN,NaN,NaN,NaN,NaN
16,She,NaN,NaN,NaN,NaN,NaN,clergywoman,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,NaN,NaN,NaN,name_female_1,NaN,NaN,NaN,NaN,NaN,she,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8406,name_female_1,NaN,NaN,NaN,name_female_2,name_female_3,surname_1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8411,NaN,NaN,NaN,name_female_1,surname_1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8416,name_female_1,NaN,mother,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8426,NaN,sister,name_female_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
def change_words(original_phrase,ground_truth):
    new_phrase=original_phrase.copy()
    for j in list(ground_truth.index):
        if not pd.isna(ground_truth.loc[j]):
            new_phrase.loc[j]=ground_truth.loc[j]
    return list(new_phrase)

In [9]:
all_fem=[]
all_male=[]
subj_fem=[]
subj_male=[]
for idx in list(original_phrases.index):
    all_fem_phrase=change_words(original_phrases.loc[idx],all_female_phrases_gt.loc[idx+1])
    all_fem.append(all_fem_phrase)
    
    all_male_phrase=change_words(original_phrases.loc[idx],all_male_phrases_gt.loc[idx+2])
    all_male.append(all_male_phrase)
    
    subj_fem_phrase=change_words(original_phrases.loc[idx],female_sub_phrases_gt.loc[idx+3])
    subj_fem.append(subj_fem_phrase)
    
    subj_male_phrase=change_words(original_phrases.loc[idx],male_sub_phrases_gt.loc[idx+4])
    subj_male.append(subj_male_phrase)
    
all_fem=pd.DataFrame(all_fem)
all_male=pd.DataFrame(all_male)
subj_fem=pd.DataFrame(subj_fem)
subj_male=pd.DataFrame(subj_male)

In [10]:
all_fem

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,Shortly,afterwards,",",she,visits,Netherfield,",",Mrs.,surname_1,'s,...,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,She,appears,attracted,to,name_female_1,surname_1,(,the,eldest,surname_1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,She,declines,to,dance,with,name_female_1,(,the,second-eldest,surname_1,...,enough,to,tempt,her,.,NaN,NaN,NaN,NaN,NaN
3,She,is,a,pompous,",",obsequious,clergywoman,who,intends,to,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,After,learning,that,name_female_1,may,soon,be,engaged,",",she,...,both,age,and,beauty,.,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1605,name_female_1,'s,mother,",",name_female_2,name_female_3,surname_1,",",was,an,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1606,A,girl,named,name_female_1,surname_1,was,the,unrequited,love,of,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1607,name_female_1,'s,mother,gets,into,debt,and,the,family,leaves,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1608,Her,sister,name_female_1,notices,that,she,is,less,lively,after,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
all_male

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,Shortly,afterwards,",",he,visits,Netherfield,",",Mr.,surname_1,'s,...,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,He,appears,attracted,to,name_male_1,surname_1,(,the,eldest,surname_1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,He,declines,to,dance,with,name_male_1,(,the,second-eldest,surname_1,...,enough,to,tempt,him,.,NaN,NaN,NaN,NaN,NaN
3,He,is,a,pompous,",",obsequious,clergyman,who,intends,to,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,After,learning,that,name_male_1,may,soon,be,engaged,",",he,...,both,age,and,beauty,.,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1605,name_male_1,'s,father,",",name_male_2,name_male_3,surname_1,",",was,an,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1606,A,boy,named,name_male_1,surname_1,was,the,unrequited,love,of,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1607,name_male_1,'s,father,gets,into,debt,and,the,family,leaves,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1608,His,brother,name_male_1,notices,that,he,is,less,lively,after,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
subj_fem

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,Shortly,afterwards,",",she,visits,Netherfield,",",Mr.,surname_1,'s,...,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,She,appears,attracted,to,name_female_1,surname_1,(,the,eldest,surname_1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,She,declines,to,dance,with,name_female_1,(,the,second-eldest,surname_1,...,enough,to,tempt,him,.,NaN,NaN,NaN,NaN,NaN
3,She,is,a,pompous,",",obsequious,clergywoman,who,intends,to,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,After,learning,that,name_female_1,may,soon,be,engaged,",",she,...,both,age,and,beauty,.,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1605,name_male_1,'s,mother,",",name_female_2,name_female_3,surname_1,",",was,an,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1606,A,girl,named,name_female_1,surname_1,was,the,unrequited,love,of,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1607,name_male_1,'s,mother,gets,into,debt,and,the,family,leaves,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1608,Her,sister,name_female_1,notices,that,she,is,less,lively,after,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
subj_male

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,Shortly,afterwards,",",he,visits,Netherfield,",",Mr.,surname_1,'s,...,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,He,appears,attracted,to,name_female_1,surname_1,(,the,eldest,surname_1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,He,declines,to,dance,with,name_female_1,(,the,second-eldest,surname_1,...,enough,to,tempt,him,.,NaN,NaN,NaN,NaN,NaN
3,He,is,a,pompous,",",obsequious,clergyman,who,intends,to,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,After,learning,that,name_female_1,may,soon,be,engaged,",",he,...,both,age,and,beauty,.,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1605,name_male_1,'s,father,",",name_male_2,name_male_3,surname_1,",",was,an,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1606,A,boy,named,name_male_1,surname_1,was,the,unrequited,love,of,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1607,name_male_1,'s,father,gets,into,debt,and,the,family,leaves,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1608,Her,brother,name_male_1,notices,that,she,is,less,lively,after,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Changes to the names and surnames
+ for the surnames I deleted the ones that had "'" such as O'brain (that's why there are only 248 surnames and not 250)
+ for both I deleted the (sur)names that ended in "s"

In [14]:
female_names = pd.read_csv('Top250Female1996-2019.txt', sep="\n", header=None)
male_names = pd.read_csv('Top250Male1996-2019.txt', sep="\n", header=None)
surnames = pd.read_csv('Top250Surnames1991-2000.txt', sep="\n", header=None) 

print(female_names.shape,male_names.shape,surnames.shape)


(250, 1) (250, 1) (248, 1)


In [15]:
female_names=pd.DataFrame([name for name in list(female_names[0]) if name[-1]!='s'])
male_names=pd.DataFrame([name for name in list(male_names[0]) if name[-1]!='s'])
surnames=pd.DataFrame([name for name in list(surnames[0]) if (name[-1]!='s' or name.find("'") != -1)])

female_names=list(name[0] for name in female_names.values)
male_names=list(name[0] for name in male_names.values)
surnames=list(name[0] for name in surnames.values)

print(len(female_names),len(male_names),len(surnames))

245 231 193


In [16]:

def change_names(phrase):
    p=phrase.copy()

    for word_idx in list(p.index):
        if str(p.loc[word_idx])[:9]=='name_male':
            p.loc[word_idx]=male_names[int(p.loc[word_idx][-1])-1] #in these phrases the biggest amount of different names is 4 but if there are more than 9, use the str.rfind
            #print(1)
        elif str(p.loc[word_idx])[:11]=='name_female':
            p.loc[word_idx]=female_names[int(p.loc[word_idx][-1])-1]
            #print(2)
        elif str(p.loc[word_idx])[:7]=='surname':
            if p.loc[word_idx][-2:]=='pl':
                p.loc[word_idx]=surnames[int(p.loc[word_idx][-4])-1]+'s'
             #   print(3)
            else:
                p.loc[word_idx]=surnames[int(p.loc[word_idx][-1])-1]
              #  print(4)
    #print(p)
    return p


In [17]:
final_all_fem=[]
final_all_male=[]
final_subj_fem=[]
final_subj_male=[]
for idx in all_fem.index:
    random.shuffle(female_names)
    random.shuffle(male_names)
    random.shuffle(surnames)
    
    phrase1=change_names(all_fem.iloc[idx])
    final_all_fem.append(phrase1)
    
    phrase2=change_names(all_male.iloc[idx])
    final_all_male.append(phrase2)
    
    phrase3=change_names(subj_fem.iloc[idx])
    final_subj_fem.append(phrase3)
    
    phrase4=change_names(subj_male.iloc[idx])
    final_subj_male.append(phrase4)
    

In [18]:
final_all_fem=pd.DataFrame(final_all_fem)
final_all_fem

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,Shortly,afterwards,",",she,visits,Netherfield,",",Mrs.,Bradley,'s,...,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,She,appears,attracted,to,Lucie,Scott,(,the,eldest,Scott,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,She,declines,to,dance,with,Maisie,(,the,second-eldest,Sullivan,...,enough,to,tempt,her,.,NaN,NaN,NaN,NaN,NaN
3,She,is,a,pompous,",",obsequious,clergywoman,who,intends,to,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,After,learning,that,Jodie,may,soon,be,engaged,",",she,...,both,age,and,beauty,.,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1605,Ava,'s,mother,",",Milly,Mya,Walton,",",was,an,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1606,A,girl,named,Edie,Sutton,was,the,unrequited,love,of,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1607,Lyla,'s,mother,gets,into,debt,and,the,family,leaves,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1608,Her,sister,Kate,notices,that,she,is,less,lively,after,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
final_all_male=pd.DataFrame(final_all_male)
final_all_male

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,Shortly,afterwards,",",he,visits,Netherfield,",",Mr.,Bradley,'s,...,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,He,appears,attracted,to,Jakub,Scott,(,the,eldest,Scott,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,He,declines,to,dance,with,Caleb,(,the,second-eldest,Sullivan,...,enough,to,tempt,him,.,NaN,NaN,NaN,NaN,NaN
3,He,is,a,pompous,",",obsequious,clergyman,who,intends,to,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,After,learning,that,Freddie,may,soon,be,engaged,",",he,...,both,age,and,beauty,.,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1605,Adam,'s,father,",",Scott,Robbie,Walton,",",was,an,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1606,A,boy,named,Stephen,Sutton,was,the,unrequited,love,of,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1607,William,'s,father,gets,into,debt,and,the,family,leaves,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1608,His,brother,Matthew,notices,that,he,is,less,lively,after,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
final_subj_fem=pd.DataFrame(final_subj_fem)
final_subj_fem

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,Shortly,afterwards,",",she,visits,Netherfield,",",Mr.,Bradley,'s,...,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,She,appears,attracted,to,Lucie,Scott,(,the,eldest,Scott,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,She,declines,to,dance,with,Maisie,(,the,second-eldest,Sullivan,...,enough,to,tempt,him,.,NaN,NaN,NaN,NaN,NaN
3,She,is,a,pompous,",",obsequious,clergywoman,who,intends,to,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,After,learning,that,Jodie,may,soon,be,engaged,",",she,...,both,age,and,beauty,.,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1605,Adam,'s,mother,",",Milly,Mya,Walton,",",was,an,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1606,A,girl,named,Edie,Sutton,was,the,unrequited,love,of,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1607,William,'s,mother,gets,into,debt,and,the,family,leaves,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1608,Her,sister,Kate,notices,that,she,is,less,lively,after,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
final_subj_male=pd.DataFrame(final_subj_male)
final_subj_male

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,Shortly,afterwards,",",he,visits,Netherfield,",",Mr.,Bradley,'s,...,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,He,appears,attracted,to,Lucie,Scott,(,the,eldest,Scott,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,He,declines,to,dance,with,Maisie,(,the,second-eldest,Sullivan,...,enough,to,tempt,him,.,NaN,NaN,NaN,NaN,NaN
3,He,is,a,pompous,",",obsequious,clergyman,who,intends,to,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,After,learning,that,Jodie,may,soon,be,engaged,",",he,...,both,age,and,beauty,.,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1605,Adam,'s,father,",",Scott,Robbie,Walton,",",was,an,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1606,A,boy,named,Stephen,Sutton,was,the,unrequited,love,of,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1607,William,'s,father,gets,into,debt,and,the,family,leaves,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1608,Her,brother,Matthew,notices,that,she,is,less,lively,after,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Creating the two datasets
+ There are two datasets:
    + one where the sentences are completly female or completly male
    + one where the subject (and words related to it) are female or male

+ For each dataset randomly half of the indexes are chosen to be male and the other half female and these are the phrases that will have a target value of 0 for female and 1 for male.

### All same gender
+ from the datasets:
    + final_all_male
    + final_all_female
+ to the dataset:
    + data_all_same

+ Creating datasets:
    + receives a dataframe of phrases/tokens
    + chooses the indexes that will be male
    + adds a column in the beginning with the target value
    + saves the dataset
    + returns dataset and male indexes

In [24]:

def Create_Dataset(dataframe_female,dataframe_male,save=True,name=''):
    #randomly choosing the indexes that will be male (target=1)
    indexes=list(range(len(dataframe_male)))
    indexes_male=random.sample(indexes, k = int(len(indexes)/2))
    indexes_female=[i for i in list(range(len(dataframe_male))) if i not in indexes_male]

    #choosing the phrases
    df_male=dataframe_male.iloc[indexes_male]
    df_female=dataframe_female.iloc[indexes_female]
    
    #adding the target column (1st column)
    df_male.insert(0, 'target', np.ones(len(indexes_male)))
    df_female.insert(0, 'target', np.zeros(len(indexes_male)))

    #concatenate dataframes to create dataset
    df= pd.concat([df_male,df_female])
    df=df.sort_index()
    
    if save:  
        with open(name+'.pkl', 'wb') as f:
            pickle.dump(df,f)
    
    return df, indexes_male, indexes_female

In [25]:
Create_Dataset(final_all_fem,final_all_male,name='data_all_same')
Create_Dataset(final_subj_fem,final_subj_male,name='data_subj')

(      target        0           1          2        3         4            5  \
 0        0.0  Shortly  afterwards          ,      she    visits  Netherfield   
 1        1.0       He     appears  attracted       to     Lucie        Scott   
 2        1.0       He    declines         to    dance      with       Maisie   
 3        0.0      She          is          a  pompous         ,   obsequious   
 4        1.0    After    learning       that    Jodie       may         soon   
 ...      ...      ...         ...        ...      ...       ...          ...   
 1605     0.0     Adam          's     mother        ,     Milly          Mya   
 1606     0.0        A        girl      named     Edie    Sutton          was   
 1607     1.0  William          's     father     gets      into         debt   
 1608     1.0      Her     brother    Matthew  notices      that          she   
 1609     1.0      His     husband      tends       to  Florence            ,   
 
                 6        

In [26]:
#randomly choosing the indexes that will be male (target=1)
indexes=list(range(len(final_subj_male)))
indexes_male=random.sample(indexes, k = int(len(indexes)/2))

#creating the target
target=np.zeros((len(final_subj_male)))
target[indexes_male]=1

#choosing the phrases
data_all_same=[]
for i in range(len(final_subj_male)):
    if i in indexes_male:
        data_all_same.append(final_all_male.loc[i])
    else:
        data_all_same.append(final_all_fem.loc[i])
data_all_same=pd.DataFrame(data_all_same)
data_all_same

#saving the data_set+target
data=[[data_all_same.loc[i],target[i]] for i in range(len(target))]

with open('data_all_same.pkl', 'wb') as f:
    pickle.dump(data,f)

### Only the subject is the same gender
+ from the datasets:
    + final_subj_male
    + final_subj_fem
+ to the dataset:
    + data_subj

In [27]:
#randomly choosing the indexes that will be male (target=1)
indexes=list(range(len(final_subj_male)))
indexes_male=random.sample(indexes, k = int(len(indexes)/2))

#creating the target
target=np.zeros((len(final_subj_male)))
target[indexes_male]=1

#choosing the phrases
data_subj=[]
for i in range(len(final_subj_male)):
    if i in indexes_male:
        data_subj.append(final_subj_male.loc[i])
    else:
        data_subj.append(final_subj_fem.loc[i])
        
data_subj=pd.DataFrame(data_subj)

#saving the data_set+target
data=[[data_subj.loc[i],target[i]] for i in range(len(target))]

with open('data_subj.pkl', 'wb') as f:
    pickle.dump(data,f)

In [28]:
dataframe_male=final_all_male
dataframe_female=final_all_fem

#randomly choosing the indexes that will be male (target=1)
indexes=list(range(len(dataframe_male)))
indexes_male=random.sample(indexes, k = int(len(indexes)/2))
indexes_female=[i for i in list(range(len(dataframe_male))) if i not in indexes_male]

#creating the target
target=np.zeros((len(final_subj_male)))
target[indexes_male]=1

#choosing the phrases
df_male=dataframe_male.iloc[indexes_male]
df_male.insert(0, 'target', np.ones(len(indexes_male)))
df_female=dataframe_female.iloc[indexes_female]
df_female.insert(0, 'target', np.zeros(len(indexes_male)))

df= pd.concat([df_male,df_female])
df=df.sort_index()
df

,target,0,1,2,3,4,5,6,7,8,...,19,20,21,22,23,24,25,26,27,28
0,1.0,Shortly,afterwards,",",he,visits,Netherfield,",",Mr.,Bradley,...,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,She,appears,attracted,to,Lucie,Scott,(,the,eldest,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,He,declines,to,dance,with,Caleb,(,the,second-eldest,...,enough,to,tempt,him,.,NaN,NaN,NaN,NaN,NaN
3,1.0,He,is,a,pompous,",",obsequious,clergyman,who,intends,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,After,learning,that,Freddie,may,soon,be,engaged,",",...,both,age,and,beauty,.,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1605,1.0,Adam,'s,father,",",Scott,Robbie,Walton,",",was,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1606,0.0,A,girl,named,Edie,Sutton,was,the,unrequited,love,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1607,1.0,William,'s,father,gets,into,debt,and,the,family,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1608,0.0,Her,sister,Kate,notices,that,she,is,less,lively,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
len(indexes_female)

805